In [0]:
json_sample = """
[
    {
      "id": "0001",
      "FirstName": "Sonalika",
      "LastName": "kishan",
      "Qualification": "Masters",
      "Technology": "ADF",
      "Age": 34,
      "Address": [
                    {
                        "city": "Bangalore",
                        "HouseAndPIN": [
                            [
                                "#234",
                                "Agara",
                                "Bangalore",
                                "Karnataka"
                            ],
                            [
                                "560105",
                                "560107"
                            ]
                        ]
                    },
                    {
                        "city": "Chennai",
                        "HouseAndPIN": [
                            [
                                "#569",
                                "Velachery",
                                "Chennai",
                                "Tamilnadu"
                            ],
                            [
                                "405603",
                                "408613"
                            ]
                        ]
                    }
                ]
    },
    {
      "id": "0002",
      "FirstName": "Santhosh",
      "LastName": "kumar",
      "Qualification": "Masters",
      "Technology": "Azure",
      "Age": 44,
      "Address": [
                    {
                        "city": "Hyderabad",
                        "HouseAndPIN": [
                            [
                                "667",
                                "Jublihills",
                                "Hyderabad"
                            ],
                            [
                              "711178",
                              "711354"
                            ]
                        ]
                    },
                    {
                        "city": "Amaravati",
                        "HouseAndPIN": [
                            [
                                "512",
                                "BenzCircle",
                                "Vijayawada",
                                "Amaravati"
                            ],
                            [
                                "560238",
                                "577034",
                                "678019",
                                "778899"
                            ]
                        ]
                    }
                ]
    },
    {
      "id": "0003",
      "FirstName": "Sourabh",
      "LastName": "Geol",
      "Qualification": "Bachelers",
      "Technology": "AWS",
      "Age": 28,
      "Address": [
                    {
                        "city": "Kolkata",
                        "HouseAndPIN": [
                            [
                                "4453",
                                "west bengal"
                            ],
                            [
                              "710123",
                              "710176"
                            ]
                        ]
                    },
                    {
                        "city": "Sweden",
                        "HouseAndPIN": [
                            [
                                "1CACV",
                                "Gothenberg",
                                "sweden"
                            ],
                            [
                              "DAFEC1A",
                              "DAFED2B",
                              "DAFEE33"
                            ]
                        ]
                    }
                ]
    }
]
"""

dbutils.fs.put("/FileStore/tables/Flatten Nested Array.json", json_sample, True)

In [0]:
from pyspark.sql.types import *

Schema = StructType([
    StructField("id", StringType(), True),
    StructField("FirstName", StringType(), True),
    StructField("LastName", StringType(), True),
    StructField("Qualification", StringType(), True),
    StructField("Technology", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Address", ArrayType(
      StructType([
        StructField("city", StringType(), True),
        StructField("HouseAndPIN", ArrayType(ArrayType(StringType())), True)
      ])
    ), True)
])

In [0]:
import json
dfPerson = json.loads(json_sample)
dfPerson = spark.createDataFrame(dfPerson, schema = Schema)
display(dfPerson)
dfPerson.printSchema()

- Flatten the “home” Column to Have **“All” the “Array Elements”** in a **“Single Array Column”** for **“Each City”** in the New Row Using the **“flatten()”** Method.

In [0]:
from pyspark.sql.functions import *

json_dfPerson = dfPerson.select(
                                col("FirstName"),
                                col("LastName"),
                                col("Age"),
                                col("Address.city").alias("City"),
                                flatten(col("Address.HouseAndPIN")).alias("flattenedHouseAndPIN")
                                )
display(json_dfPerson)

# Print the "Schema" of the Created DataFrame
json_dfPerson.printSchema()

**EX 01**

**How to flatten Array of Array using flatten function?**

In [0]:
from pyspark.sql.functions import col, explode, posexplode, flatten

data = [("Revanth", [["ADF", "Spark", "ADB"], ["ETL", "Devops", None], ["SQL", None]]),
        ("Reshma", [["SSMS", None, "Salesforce"], ["SAP", "ERP", None]]),
        ("Raashi", [["Python" "VB", None], ["C++", "GitHub", "Git"]]),
        ("Krishna", [["SHELL", "DRG"], ["JAVA", None]]),
        ("Sudarshan", None),
        ("Kamal", [])
       ]

columns = ["EmpName", "Technology"]

df = spark.createDataFrame(data=data, schema=columns)
display(df)
df.printSchema()

EmpName,Technology
Revanth,"List(List(ADF, Spark, ADB), List(ETL, Devops, null), List(SQL, null))"
Reshma,"List(List(SSMS, null, Salesforce), List(SAP, ERP, null))"
Raashi,"List(List(PythonVB, null), List(C++, GitHub, Git))"
Krishna,"List(List(SHELL, DRG), List(JAVA, null))"
Sudarshan,null
Kamal,List()


root
 |-- EmpName: string (nullable = true)
 |-- Technology: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)



In [0]:
display(df.select("EmpName", flatten("Technology").alias('NewTechnology')))

EmpName,NewTechnology
Revanth,"List(ADF, Spark, ADB, ETL, Devops, null, SQL, null)"
Reshma,"List(SSMS, null, Salesforce, SAP, ERP, null)"
Raashi,"List(PythonVB, null, C++, GitHub, Git)"
Krishna,"List(SHELL, DRG, JAVA, null)"
Sudarshan,null
Kamal,List()


In [0]:
dfff=df.select("EmpName", flatten("Technology").alias('NewTechnology'))
display(dfff)

EmpName,NewTechnology
Revanth,"List(ADF, Spark, ADB, ETL, Devops, null, SQL, null)"
Reshma,"List(SSMS, null, Salesforce, SAP, ERP, null)"
Raashi,"List(PythonVB, null, C++, GitHub, Git)"
Krishna,"List(SHELL, DRG, JAVA, null)"
Sudarshan,null
Kamal,List()


In [0]:
df5 = dfff.select("EmpName", explode("NewTechnology").alias('CoreTechnology'))
display(df5)

EmpName,CoreTechnology
Revanth,ADF
Revanth,Spark
Revanth,ADB
Revanth,ETL
Revanth,Devops
Revanth,null
Revanth,SQL
Revanth,null
Reshma,SSMS
Reshma,null


**EX 02**

**How to flatten Array of Array using Explode function?**

In [0]:
from pyspark.sql.functions import col, explode, posexplode, flatten

data1 = [("Revanth", [["ADF", "Spark", "ADB"], ["ETL", "Devops", None], ["SQL", None], None]),
        ("Reshma", [["SSMS", None, "Salesforce"], ["SAP", "ERP", None], None]),
        ("Raashi", [["Python" "VB", None], ["C++", "GitHub", "Git"]]),
        ("Krishna", [["SHELL", "DRG"], ["JAVA", None]]),
        ("Sudarshan", None),
        ("Kamal", [])
       ]

columns1 = ["EmpName", "Technology"]

df1 = spark.createDataFrame(data=data1, schema=columns1)
display(df1)
df1.printSchema()

EmpName,Technology
Revanth,"List(List(ADF, Spark, ADB), List(ETL, Devops, null), List(SQL, null), null)"
Reshma,"List(List(SSMS, null, Salesforce), List(SAP, ERP, null), null)"
Raashi,"List(List(PythonVB, null), List(C++, GitHub, Git))"
Krishna,"List(List(SHELL, DRG), List(JAVA, null))"
Sudarshan,null
Kamal,List()


root
 |-- EmpName: string (nullable = true)
 |-- Technology: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)



In [0]:
df2 = df1.select("EmpName", explode("Technology").alias('NewTechnology'))
display(df2)

EmpName,NewTechnology
Revanth,"List(ADF, Spark, ADB)"
Revanth,"List(ETL, Devops, null)"
Revanth,"List(SQL, null)"
Revanth,null
Reshma,"List(SSMS, null, Salesforce)"
Reshma,"List(SAP, ERP, null)"
Reshma,null
Raashi,"List(PythonVB, null)"
Raashi,"List(C++, GitHub, Git)"
Krishna,"List(SHELL, DRG)"


In [0]:
df3 = df2.select("EmpName", explode("NewTechnology").alias('CoreTechnology'))
display(df3)

EmpName,CoreTechnology
Revanth,ADF
Revanth,Spark
Revanth,ADB
Revanth,ETL
Revanth,Devops
Revanth,null
Revanth,SQL
Revanth,null
Reshma,SSMS
Reshma,null
